In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Literal
from scipy.stats import pearsonr

# Preprocessing

In [6]:
# 类：数据处理与分析

class TrajAnalysor():
    """数据处理与分析"""

    # 初始化
    def __init__(self, city: Literal['A', 'B', 'C', 'D']):
        self.city = city
        self.dataframe = pd.read_csv(self.get_path(), compression='gzip')
        self.masked_dataframe = self.dataframe
        self.unmasked_days = 75

    # 获取数据集路径
    def get_path(self):
        path_dict = {
            'A': './dataset/cityA_groundtruthdata.csv.gz',
            'B': './dataset/cityB_challengedata.csv.gz',
            'C': './dataset/cityC_challengedata.csv.gz',
            'D': './dataset/cityD_challengedata.csv.gz'
        }
        return path_dict.get(self.city, "Invalid City!")
    
    # 剔除数据
    def reinitial(self):
        """再初始化数据"""
        self.masked_dataframe = self.dataframe

    def mask_users(self, reverse=False):
        """剔除待预测用户"""
        if self.city in ['B', 'C', 'D']:
            if reverse:
                df = self.masked_dataframe[self.masked_dataframe['uid'] >= len(pd.unique[self.dataframe['uid']]) - 3000]
            else:
                df = self.masked_dataframe[self.masked_dataframe['uid'] < len(pd.unique[self.dataframe['uid']]) - 3000]
        self.masked_dataframe = df
    
    def mask_points(self, reverse=False):
        """剔除待预测数据点"""
        if reverse:
            df = self.masked_dataframe[self.dataframe['x'] == 999]
        else:
            df = self.masked_dataframe[self.dataframe['x'] != 999]
        self.masked_dataframe = df

    def mask_days(self, reverse=False):
        """剔除待预测区间"""
        if reverse:
            df = self.masked_dataframe[self.masked_dataframe['day'] < 60]
        else:
            df = self.masked_dataframe[self.masked_dataframe['day'] >= 60]
        self.masked_dataframe = df


    # 距离统计量
    def euclidean_distance(self, group, is_sum=True):
        """辅助函数：计算欧式距离"""
        distances = np.sqrt(np.diff(group['x'])**2 + np.diff(group['y'])**2)
        if is_sum:
            return distances.sum()
        else:
            return distances
    
    def distance_peruser(self):
        dpu = self.masked_dataframe.groupby('uid').apply(self.euclidean_distance)
        return dpu, 'dpu'
    
    def distance_peruser_perday(self):
        dpp = self.mask_dataframe.groupby(['uid', 'day']).apply(self.euclidean_distance)
        return dpp, 'dpp'
        
    def distance_perstep(self):
        dps = []
        for _, traj in self.mask_dataframe.groupby('uid'):
            distances = self.euclidean_distance(traj, is_sum=False)
            dps.extend(distances)
        dps = np.array(dps)
        return dps, 'dps'

    # 计数统计量
    def count_peruser(self):
        cpu = self.masked_dataframe.groupby('uid').size()
        return cpu.to_numpy(), 'cpu'
    
    def count_peruser_perday(self):
        cpp = self.mask_dataframe.groupby(['uid', 'day']).size()
        return cpp.to_numpy(), 'cpp'
    
    # 时间统计量
    def histogram_data_count(self):
        days = 75
        timeslots = 48  
        timestamp_counts = np.zeros(days * timeslots, dtype=int)

        for _, row in self.mask_dataframe.iterrows():
            day = row['day']
            timeslot = row['timeslot']
            index = day * timeslots + timeslot
            timestamp_counts[index] += 1
        
        return timestamp_counts
    
    def histogram_move_count(self):
        days = 75
        timeslots = 48  
        timestamp_counts = np.zeros(days * timeslots, dtype=int)

        for _, traj in self.mask_dataframe.groupby('uid'):
            x_prev = traj[0]['x']
            y_prev = traj[0]['y']
            for _, row in traj.iterrows():
                if (row['x'] != x_prev) or (row['y'] != y_prev):
                    day = row['day']
                    timeslot = row['timeslot']
                    index = day * timeslots + timeslot
                    timestamp_counts[index] += 1
                x_prev = row['x']
                y_prev = row['y']
    
        return timestamp_counts
    
    #绘制各种图像

    def get_data(self, type):
        """获取统计数据"""
        type_dict = {
            'dpu': self.distance_peruser,
            'dpp': self.distance_peruser_perday,
            'dps': self.distance_perstep,
            'cpu': self.count_peruser,
            'cpp': self.count_peruser_perday,
        }
        return type_dict(type, "Invalid type!")
    
    def plot_bar(self,
                 type: Literal['dpu', 'dpp', 'dps', 'cpu', 'cpp'],
                 binstamp=100, total_days=75,
                 freq_normalize=True, dis_normalize=False):
        """绘制数据分布柱状图，频数及距离可归一化"""
        
        # 标题和标签字典
        def switch_title(type, city, total_days):
            title_dict = {
                'dpu': f'Distribution of distance/user during 1-{str(total_days)} day for city {city}',
                'dpp': f'Distribution of distance/(user*day) during 1-{str(total_days)} day for city {city}',
                'dps': f'Distribution of distance/step during 1-{str(total_days)} day for city {city}',
                'cpu': f'Distribution of counts/user for city {city}',
                'cpp': f'Distribution of counts/(user*day) for city {city}',
                }
            xlabel_dict = {
                'dpu': 'distance',
                'dpp': 'distance',
                'dps': 'distance',
                'cpu': 'count',
                'cpp': 'count',
            }
            return title_dict.get(type, "Invalid type!"), xlabel_dict.get(type, "Invalid type!")

        # 柱状图参数
        data_array = self.get_data(type)
        counts, bins = np.histogram(data_array, bins=binstamp)
        title, x_label = switch_title(type, self.city, total_days)
        y_label = 'Frequency'

        # 计算平均数,中位数,最大值和最小值"
        mean = np.mean(data_array)
        median = np.median(data_array)
        max = np.max(data_array)
        min = np.min(data_array)

        # 归一化
        if freq_normalize:
            total_counts = np.sum(counts)
            counts = counts / total_counts
            y_label = y_label + f'(normalized, max= {str(total_counts)})'
        
        if dis_normalize:
            total_dis = bins[-1]
            bins[:] = bins[:] / total_dis
            mean, median = mean / total_dis, median / total_dis
            x_label = x_label + f'(normalized, max= {str(total_dis)})'

        # 绘制柱状图
        plt.figure()
        plt.bar(bins[:-1], counts, width=np.diff(bins), edgecolor='black', alpha=0.7)

        # 标注平均数
        plt.axvline(mean, color='red', linestyle='--', linewidth=1, label=f'Mean: {mean:.3f}')
        plt.text(mean, plt.ylim()[1] * 0.9, f'Mean: {mean:.3f}', color='red', ha='left', va='center')
        # 标注中位数
        plt.axvline(median, color='green', linestyle='--', linewidth=1, label=f'Median: {median:.3f}')
        plt.text(median, plt.ylim()[1] * 0.8, f'Median: {median:.3f}', color='green', ha='left', va='center')
        # 添加最大值的标注
        plt.axvline(max, color='black', linestyle='--', linewidth=1, label=f'Max: {max:.3f}')
        plt.text(max, plt.ylim()[1] * 0.6, f'Max: {max:.3f}', color='black', ha='left', va='center')
        # 添加最小值的标注
        plt.axvline(min, color='black', linestyle='--', linewidth=1, label=f'Min: {min:.3f}')
        plt.text(min, plt.ylim()[1] * 0.6, f'Min: {min:.3f}', color='black', ha='left', va='center')

        # 标题和标签
        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)

        # 显示图像
        plt.show()
    
    # 函数：绘制折线图

    def plot_histogram(self,
                       timeslot_count,
                       city: Literal['A', 'B', 'C', 'D'],
                       type: Literal['Data', 'Move']):
        """绘制单城市时间轴折线图"""
        # 绘制折线图
        plt.figure(figsize=(80, 5))
        plt.plot(timeslot_count, linestyle='-', color='blue')

        # 设置坐标轴范围
        plt.xlim(0, len(timeslot_count) - 1)  # 使 x 轴范围覆盖整个数据
        plt.ylim(min(timeslot_count), max([max(timeslot_count), 1000]))  # 使 y 轴范围覆盖所有数据点

        # 生成连续的时间序列刻度并设置刻度标签
        x_ticks = np.arange(0, (75 + 1) * 48, 48)  # 每48个timeslot为一天
        x_tick_labels = [f'Day {i}' for i in range(75 + 1)]
        plt.xticks(ticks=x_ticks, labels=x_tick_labels, rotation=30)

        # 标题标签
        plt.title(f'{type} Counts Histogram for city {city}')
        plt.xlabel('Day')
        plt.ylabel('Counts')

        # 显示图像
        plt.show()

    
        

In [2]:
# 函数：绘制折线图

def plot_histogram(timeslot_count,
                   city: Literal['A', 'B', 'C', 'D'],
                   type: Literal['Data', 'Move']):
    """绘制单城市时间轴折线图"""
    # 绘制折线图
    plt.figure(figsize=(80, 5))
    plt.plot(timeslot_count, linestyle='-', color='blue')

    # 设置坐标轴范围
    plt.xlim(0, len(timeslot_count) - 1)  # 使 x 轴范围覆盖整个数据
    plt.ylim(min(timeslot_count), max([max(timeslot_count), 1000]))  # 使 y 轴范围覆盖所有数据点

    # 生成连续的时间序列刻度并设置刻度标签
    x_ticks = np.arange(0, (75 + 1) * 48, 48)  # 每48个timeslot为一天
    x_tick_labels = [f'Day {i}' for i in range(75 + 1)]
    plt.xticks(ticks=x_ticks, labels=x_tick_labels, rotation=30)

    # 标题标签
    plt.title(f'{type} Counts Histogram for city {city}')
    plt.xlabel('Day')
    plt.ylabel('Counts')

    # 显示图像
    plt.show()


def plot_histograms(*arrs, type: Literal['Data', 'Move'], log=False):
    """绘制多城市时间轴折线图"""
    cities = ['A','B','C','D']
    colors = ['red', 'blue', 'green', 'orange']

    plt.figure(figsize=(80, 5))
    for i, arr in enumerate(arrs):
        if log:
            arr = np.log1p(arr)
        color = colors[i % len(colors)]
        plt.plot(arr, label=f'City {cities[i]}', color=color)

    # 添加图例
    plt.legend()

    # 设置坐标轴范围
    plt.xlim(0, len(arr) - 1)  # 使 x 轴范围覆盖整个数据

    # 生成连续的时间序列刻度并设置刻度标签
    x_ticks = np.arange(0, (75 + 1) * 48, 48)  # 每48个timeslot为一天
    x_tick_labels = [f'Day {i}' for i in range(75 + 1)]
    plt.xticks(ticks=x_ticks, labels=x_tick_labels, rotation=30)

    # 标题标签\n",
    plt.title(f'{type} Counts Histograms for each City')
    plt.xlabel('Day')
    plt.ylabel('Counts')

    # 显示图表
    plt.show()

In [4]:
# 函数：绘制相关性散点图

def plot_multi_scatter(ref_arr, *arrs,
                       type:Literal['counts', 'distance'] = 'counts'):
    num_arrs = len(arrs)
    name_list = ['A', 'B', 'C', 'D']
    plt.figure(figsize=(14, 5 * num_arrs))

    for i, series in enumerate(arrs):
        correlation, _ = pearsonr(ref_arr, series)

        plt.subplot(num_arrs, 1, i + 1)
        plt.scatter(ref_arr, series, color='blue', alpha=0.6)
        
        # 添加线性拟合直线
        m, b = np.polyfit(ref_arr, series, 1)
        plt.plot(ref_arr, m * ref_arr + b, color='red', linestyle='--', linewidth=2)

        # 设置标题和轴标签
        plt.title(f'Scatter Plot of Reference between {name_list[0]} and {name_list[i+1]} (Correlation: {correlation:.4f})')
        plt.xlabel(f'{type} for city {name_list[0]}')
        plt.ylabel(f'{type} for city {name_list[i+1]}')

    # 调整子图间距
    plt.tight_layout()
    plt.show()

In [5]:
# 函数：绘制热力图
def plot_heatmap(traj,
                 city: Literal['A', 'B', 'C', 'D'],
                 grid_sizes = [10, 20, 50],     # 热力图粒度列表
                 log=False):
    """绘制轨道数据集的不同粒度下的空间分布热力图"""

    # 提取空间信息
    x_values = traj[:, 3]
    y_values = traj[:, 4]

    # 设置子图属性
    fig, axes = plt.subplots(1, len(grid_sizes), figsize=(18, 6))

    for i, grid_size in enumerate(grid_sizes):
        # 计算每个网格的边界
        x_bins = np.linspace(np.min(x_values), np.max(x_values), grid_size)
        y_bins = np.linspace(np.min(y_values), np.max(y_values), grid_size)

        # 统计每个网格中的数据点数量
        heatmap, _, _ = np.histogram2d(x_values, y_values, bins=[x_bins, y_bins])
        if log:
            heatmap = np.log1p(heatmap)

        # 绘制热力图
        im = axes[i].imshow(heatmap.T, origin='lower', cmap='Blues', interpolation='nearest')

        # 添加标题
        axes[i].set_title(f'Grid Size: {grid_size}x{grid_size} for City {city}')
        axes[i].set_xlabel('X')
        axes[i].set_ylabel('Y')

        # 添加颜色
        plt.colorbar(im, ax=axes[i])

    # 调整布局
    plt.tight_layout()
    plt.show()

In [4]:
A = TrajAnalysor('A')
B = TrajAnalysor('B')
C = TrajAnalysor('C')
D = TrajAnalysor('D')

In [5]:
A.dataframe[:10]

,uid,d,t,x,y
0,0,0,1,79,86
1,0,0,2,79,86
2,0,0,8,77,86
3,0,0,9,77,86
4,0,0,19,81,89
5,0,0,20,82,88
6,0,0,21,81,89
7,0,0,22,81,89
8,0,0,24,76,86
9,0,0,25,75,82


# Count Statistics

## Distribution of count peruser

## Distribution of count peruser perday

# Distance Statistics

# Temporal Statistics

# Spatial Statistics